<a href="https://colab.research.google.com/github/psyrixen/Petro/blob/master/TDM_Refined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing Libraries

In [4]:
!pip install -q xlrd
#!pip install geopandas
#!pip install contextily
#!pip install shapely
#!pip install geovoronoi
!pip install sklearn
!pip install saspy

     |████████████████████████████████| 6.4MB 2.8MB/s 
  Created wheel for saspy: filename=saspy-3.7.2-cp37-none-any.whl size=6419248 sha256=fc2f70b3541a8ac6e25585ba4799e0528a1207c90a48cc8f0382af647d99359f
  Stored in directory: /root/.cache/pip/wheels/97/15/b9/04c61c8b6c0e6b28e8e8772af8b4e4b71312978e9107c3f71b
Successfully built saspy


In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
#import geopandas as gpd
#import contextily as ctx
import math 
import saspy
import warnings

In [6]:
sas = saspy.SASsession(java='/usr/bin/java', iomhost=['odaws01-apse1.oda.sas.com','odaws02-apse1.oda.sas.com'], iomport=8591, encoding='utf-8')

Using SAS Config named: default
Please enter the IOM user id: sj12031997@gmail.com
Please enter the password for IOM user : ··········
SAS Connection established. Subprocess id is 882



Reading Data File

In [7]:
from shapely.ops import cascaded_union
#from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area
#from geovoronoi import voronoi_regions_from_coords, points_to_coords
from scipy.spatial import Voronoi, voronoi_plot_2d

PStatic = pd.read_excel(open(r"/content/drive/MyDrive/Practice TDM Run on Bakken Field/Parshall Static data.xlsx",'rb'),sheet_name='Lat-Long Plot',parse_dates=True)

In [8]:
for i in range(1, 7):
  PStatic["Offset {}".format(i)] = pd.Series()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


Computing Distances for Offset Data

In [9]:
lat= PStatic.Latitude
longi =PStatic.Longitude
wellno=PStatic['NDIC #']
length = lat.size

distance_arr1=[]
distance_dict={}
offset_list=[]
offset_dict={}
dict_1={}

for well in wellno :
    latitude_1=PStatic[PStatic['NDIC #']==well]['Latitude']
    longitude_1=PStatic[PStatic['NDIC #']==well]['Longitude']
    latitude_l=latitude_1.tolist()
    longitude_l=longitude_1.tolist()
    latitude=latitude_l[0]
    longitude=longitude_l[0]

    for other_well in wellno:
      lat_2=PStatic[PStatic['NDIC #']==other_well]['Latitude']
      long_2=PStatic[PStatic['NDIC #']==other_well]['Longitude']
      distance = math.sqrt((lat_2-latitude)**2+(long_2-longitude)**2)
      dict_1[other_well]=distance
      sorting_dist=sorted(dict_1.items(),key=lambda kv:kv[1])

    for k in range(7):
      offset_list.append(sorting_dist[k])
    
    
    offset_dict[well]=offset_list
    offset_list=[]


In [21]:
key_list=offset_dict.keys()

In [42]:
for well in key_list :
  well_offsets_list=offset_dict[well]
  strw='/content/drive/MyDrive/Practice TDM Run on Bakken Field/Data Set Well Wise/'+str(well)+'.xlsx'
  well_df= pd.read_excel(open(strw,'rb'),sheet_name='Sheet1')
  loop_offsets=well_offsets_list[1:]
  counter=0
  print('Reading File Well No - ',well)
  for offset,dist in loop_offsets :
    para_list=['Distance','Pay Thickness','Permeability','Gas Injection']
    
    stro='/content/drive/MyDrive/Practice TDM Run on Bakken Field/Data Set Well Wise/'+str(offset)+'.xlsx'
    offset_well_df=pd.read_excel(open(stro,'rb'),sheet_name='Sheet1')
    pay_thickness=offset_well_df.iloc[2,20]
    permeability=offset_well_df.iloc[2,28]
    # gas_inj=offset_well_df['Gas Injection']   Add iloc index for Gas Injection once the cloumn is added
    
    well_df.iloc[:,(34+counter)]=dist
    well_df.iloc[:,(35+counter)]=pay_thickness
    well_df.iloc[:,(36+counter)]=permeability
    # well_df.iloc[:,(37+counter)]=gas_inj
    counter=counter+4
  well_df.fillna(method='ffill')
  well_df.fillna(0)
  print("Writing Well File - " , well)
  write_str="/content/drive/MyDrive/Practice TDM Run on Bakken Field/Backup Data Set Well Wise/"+str(well)+".xlsx"
  well_df.to_excel(write_str)
  print('Write Finished')
  print('---------------------------------------------------------------------------')
print('All Files Completed')

Reading File Well No -  16164
Writing Well File -  16164
Write Finished
---------------------------------------------------------------------------
Reading File Well No -  16324
Writing Well File -  16324
Write Finished
---------------------------------------------------------------------------
Reading File Well No -  16469
Writing Well File -  16469
Write Finished
---------------------------------------------------------------------------
Reading File Well No -  16484
Writing Well File -  16484
Write Finished
---------------------------------------------------------------------------
Reading File Well No -  16485
Writing Well File -  16485
Write Finished
---------------------------------------------------------------------------
Reading File Well No -  16486
Writing Well File -  16486
Write Finished
---------------------------------------------------------------------------
Reading File Well No -  16497
Writing Well File -  16497
Write Finished
----------------------------------------

#Zipping Latitude and Longitude Data

In [ ]:
zipped_lat_longi = [[lat[i], longi[i]] for i in range(0, length)]

Plotting Voronoi Diagram using 2D point Data 





In [ ]:
vor = Voronoi(zipped_lat_longi)
fig = voronoi_plot_2d(vor)
plt.figure(figsize=(25, 25))
#plt.show()

# Fuzzy Pattern Recognition


Data Preparation

In [ ]:
#initialise blank dataframe for fuzzy data
fuz = pd.read_excel(open(r"/content/drive/MyDrive/Practice TDM Run on Bakken Field/FUZZY DATAFRAME.xlsx",'rb'),sheet_name='Static Data')
 
template = pd.read_excel(open(r"/content/drive/MyDrive/Practice TDM Run on Bakken Field/FUZZY DATAFRAME.xlsx",'rb'),sheet_name='Template')
 
#create wellno string variable
path='/content/drive/MyDrive/Practice TDM Run on Bakken Field/Data Set Well Wise/'
extension='.xlsx'
 
#get well number series
well_no_series=fuz['WELL_NO']
 
#set index to prevent duplication
fuz.set_index(['WELL_NO'],inplace=True)
 
 
#fuz['WELL_NO']=(fuz['WELL_NO']).map(int)
 
#initialise blank dataframe column array
fuzz_dataf_columns = fuz.columns.values

In [ ]:
#extract data from each well file

for item in well_no_series:
  file_name = (path+(str(item))+extension)
  well_data = pd.read_excel(open(file_name,'rb'),sheet_name='Sheet1')
  #convert well file columns into template columns
  well_data.columns=template.columns
  
  #read individual columns
  for i in range(2,len(fuzz_dataf_columns)):
    name=fuzz_dataf_columns[i]
    fuz.loc[item,name]=well_data[name][0]

#reset index to restore well number column
fuz.reset_index(inplace=True) 

In [ ]:
#fuz.to_hdf('h5new.h5','/content/drive/MyDrive/Practice TDM Run on Bakken Field/')

Defining PIs

1. CUM_3_MONTH
2. CUM_6_MONTH
3. CUM_1_YR
4. CUM_3_YR
5. CUM_5_YR
6. CUM_10_YR

Scaling PIs

In [ ]:
#define PI List
#add or remove PIs accordingly, currently 6 PIs defined 

pi_list = ['CUM_3_MONTH','CUM_6_MONTH','CUM_1_YR','CUM_3_YR','CUM_5_YR','CUM_10_YR']

In [ ]:
fuz['well_fuzzy_value']=pd.Series()

Function to find roots of any given polynomial

In [ ]:
def find_roots(org_polynomial, percentiles, degree):
  
  all_roots = []
  
  for percentile in percentiles:
    polynomial=[]
    polynomial=org_polynomial
    polynomial[degree] = polynomial[degree] - percentile
    
    query = "proc iml; p = {" + ",".join(map(str, polynomial)) + "}; roots = polyroot(p);print roots;"
    final_roots = []
    roots = ((sas.submit(query, results = "TEXT"))['LST'].split("\n"))[5: -1]

    candidate_roots = []
    for item in roots :
      clean_number = item.lstrip()
      number = clean_number.split()
      if float(number[1]) == 0 :
        candidate_roots.append(float(number[0]))
      
    final_roots = [root for root in candidate_roots if root >0]
    print("root from "+str(percentile), final_roots)
    all_roots.extend(final_roots)
   
  #print(all_roots, end = ", ")
  return all_roots


Function to plot a given poly

In [ ]:
def plotf (x_coord,y_coord,ll,ul,ind) :
  bl=[]
  lower_line=[]
  upper_line=[]
  x_min=min(x_coord)
  x_max=max(x_coord)
  p=np.poly1d(np.polyfit(x_coord,y_coord,ind)) 
  xp=np.linspace(x_min-0.1,x_max+0.1,2000)
  for j in range(len(xp)):
    lower_line.append(ll)
    upper_line.append(ul)
  plt.figure()
  _=plt.plot(xp,p(xp),'*',xp,lower_line,"-",xp,upper_line,"+",linewidth=2.0)
  plt.ylim(0,1)
  bl.append(ind)
  plt.legend(bl)
  bl=[]
  plt.scatter(x_coord,y_coord)
  fig.set_size_inches(25,25)
  fig.clear(True)
  print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

Function to check if a point lies inside of a square

In [ ]:
def solve(bl, tr, p) :
   if (p[0] > bl[0] and p[0] < tr[0] and p[1] > bl[1] and p[1] < tr[1]) :
      return True
   else :
      return False

Fuzzy Logic Implementation Function

In [ ]:
def fuzzy (pi,index,limit_definition,lower,upper) :
  col_str = "pi_scaled_{0}".format(pi)
  fuz[col_str]=(fuz[pi]-min(fuz[pi]))/(max(fuz[pi])-min(fuz[pi]))
  
  x_lat=fuz['LATITUDE']
  y_lat=fuz[col_str]
  x_long=fuz['LONGITUDE']*-1
  y_long=fuz[col_str]

  #p=np.poly1d(np.polyfit(x_lat,y_lat,index)) #define polynomial for value finding
  g1=fuz[col_str].sort_values().to_list()    #sort values to find values at percentiles
  
  #define limits on graph
  if(limit_definition=="prc"):
    lower_limit = g1[int(lower * len(g1))]
    upper_limit = g1[int(upper * len(g1))]
    
  elif (limit_definition=="abs") :
    lower_limit = lower
    upper_limit = upper
    
  else :
    raise Exception("Please check limit argument")

  np.seterr(all="ignore")
  lat_poly=np.polyfit(x_lat,y_lat,index)
  long_poly=np.polyfit(x_long,y_long,index)

  poly_list = [lat_poly,long_poly]        #define iterables
  perc_list = [lower_limit ,upper_limit]  

  #find roots

  xroots = find_roots(lat_poly, perc_list, index)
  yroots = find_roots(long_poly, perc_list, index)

  
  #print(xroots,yroots)
  #fuz.to_excel("/content/drive/MyDrive/Practice TDM Run on Bakken Field/Output.xlsx")

  xroots.sort()
  yroots.sort()
  print("\n")
  #print(xroots,yroots)

  print("xroots=",xroots,"yroots=",yroots)

  # plotf(x_lat,y_lat,lower_limit,upper_limit,index)
  plotf(x_long,y_long,lower_limit,upper_limit,index)

  print("------------------------------------------------"+pi+"----------------------------------------------------------------------------------")
  # bl=[]
  # lower_line=[]
  # upper_line=[]
  # p=np.poly1d(np.polyfit(x_long,y_long,50)) 
  # xp=np.linspace(102.08,102.35,2000)
  # for j in range(len(xp)):
  #   lower_line.append(lower_limit)
  #   upper_line.append(upper_limit)
  # plt.figure()
  # _=plt.plot(xp,p(xp),'*',xp,lower_line,"-",xp,upper_line,"+")
  # plt.ylim(0,1)
  # bl.append(index)
  # plt.legend(bl)
  # bl=[]
  # plt.scatter(x_long,y_long)
  # fig.set_size_inches(25,25)
  # fig.clear(True)

  #define color dictionary
  # colors={2:'red',3:'orange',4:'yellow',5:'blue',6:'green'}


  # # define Matplotlib figure and axis
  # fig, ax = plt.subplots()

  # x_sum=0
  # y_sum=0
  # sum=0

  # #create simple line plot
  # ax.scatter(x_lat,x_long)
  # for i in range(len(xroots)-1):
  #   a = [xroots[i], xroots[i+1]]
  #   a_mid=(a[0]+a[1])/2
  #   x1=xroots[i]
  #   x2=xroots[i+1]
    
  #   x_val=np.polyval((np.poly1d(lat_poly)),a_mid)

  #   if (x_val<=lower):
  #     x_sum=1
  #   elif (x_val>lower_limit and x_val <upper_limit):
  #     x_sum=2
  #   else:
  #     x_sum=3


  #   for j in range(len(yroots)-1):
  #     b = [yroots[j],yroots[j+1]]
      
  #     y1=yroots[j]
  #     y2=yroots[j+1]

  #     bottoml=x1,y1
  #     topr=x2,y2

  #     b_mid=(b[0]+b[1])/2
  #     print("a_mid=",a_mid,"b_mid=",b_mid)
  #     y_val=np.polyval((np.poly1d(long_poly)),b_mid)
  #     print(x_val,y_val)
  #     if (y_val<=lower):
  #       y_sum=1
  #     elif (y_val>lower_limit & y_val <upper_limit):
  #       y_sum=2
  #     else:
  #       y_sum=3
      
  #     sum=x_sum+y_sum

  #     for latx,laty in arr :
  #       point=latx,laty
  #       if(solve(bottoml,topr,point)):
  #         fuz[fuz['LATITUDE']=latx and fuz['LONGITUDE']]['well_fuzzy_value']=sum

  #   #add rectangle to plot
  #     ax.add_patch(Rectangle((a[0], b[0]), (a[1] - a[0]), (b[1] - b[0]),facecolor = colors[sum],fill=True,lw=5,alpha=0.4))
  #     print(x_sum,y_sum,sum)
  # #display plot 
  # plt.xlim(47.8,48.3)
  # plt.ylim(-102.32,-102.09)
  # plt.show()
  




This code is able to produce roots for both X and Y . We need to check if the roots are OK or not . Pull MATLAB if available and run a code to check for roots. If match is found then we proceed.

In [ ]:
pi="CUM_3_MONTH"

#fuzzy(pi,24,"abs",0.19566725730255394,0.369582343949115)
fuzzy(pi,10,"abs",0.22,0.25)
  

In [ ]:
# fuz['well_fuzzy_value']

Supposing we have correct roots and clear fuzzy diagram, we will now draw the trend analysis.